In [25]:
# General libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import random
from random import randrange

In [26]:
# dictionaries with the structure source -> array of followings
dictionary = defaultdict(list)
# simple datafram to store the sources and the count of source followings
data = pd.DataFrame([])

file = open('data/train.txt', 'r') 
lines = file.readlines() 
count = 0
for line in lines: 
    count = count + 1
    split_string = list(map(int,line.strip().split("\t")))
    dictionary[split_string[0]] = []
    if (len(split_string) > 1):
        dictionary[split_string[0]] = sorted(split_string[1:len(split_string)])
        
    data = data.append(pd.DataFrame({'Source': split_string[0], 'Source_Followings': len(dictionary[split_string[0]]) }, index=[0]), ignore_index=True)

In [27]:
def followingInCommon(node1,node2):  
    list1 = dictionary[node1]
    list2 = dictionary[node2]  
    common_elements = set(list1).intersection(list2) 
    return len(common_elements)
    
# Test    
followingInCommon(3849054,161276) 

1137

In [28]:
def indirectFollowings(node_source,node_sink):
    count = 0
    list1 = dictionary[node_source]
    for node in list1:
        if node_sink in dictionary[node]:
            count += 1
    return count
    
# Test    
indirectFollowings(3849054,161276) 

67

In [23]:
indirectFollowings(161276,3849054) 

66

In [29]:
real_edges = pd.read_csv("data_generated/real_edges.csv", sep='\t')
real_edges2 = pd.read_csv("data_generated/real_edges2.csv", sep='\t')
fake_edges = pd.read_csv("data_generated/fake_edges3.csv", sep='\t')
#fake_edges2 = pd.read_csv("data_generated/fake_edges2.csv", sep='\t')

In [30]:
real_edges['Real'] = 1
real_edges2['Real'] = 1
fake_edges['Real'] = 0
#fake_edges2['Real'] = 0
frames = [real_edges, real_edges2,fake_edges]
dataset = pd.concat(frames, ignore_index=True)

In [31]:
dataset.shape

(7955, 7)

In [32]:
dataset['Common_Followings'] = dataset.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
dataset['Com_Followings_Ratio'] = dataset['Common_Followings']/dataset['Source_Followings']
dataset['Followers_Ratio'] = dataset['Sink_Followers']/dataset['Source_Followers']
dataset['Indirect_Followings'] = dataset.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
dataset['Inv_Indirect_Followings'] = dataset.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)

In [33]:
# Saving real edges
dataset.to_csv("data_models/dataset3.csv", sep='\t', index=False)

In [34]:
test_data = pd.read_csv("data_generated/test_data.csv", sep='\t')

In [35]:
test_data.head()

,Source,Sink,Source_Followings,Source_Followers,Sink_Followers,Distance
0,3563811,3600160,21,3,29,4.0
1,2052043,1401960,71,13,9,3.0
2,4517994,1690636,205,80,17,2.0
3,1660006,4349447,506,32,36,2.0
4,581111,1882617,18,5,46,3.0


In [10]:
count = 0
for index, row in test_data.iterrows():
    
    if row['Sink'] in data.Source.values:
        count += 1
        
print(count)

376


In [36]:
test_data['Common_Followings'] = test_data.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
test_data['Com_Followings_Ratio'] = test_data['Common_Followings']/test_data['Source_Followings']
test_data['Followers_Ratio'] = test_data['Sink_Followers']/test_data['Source_Followers']

In [37]:
test_data['Indirect_Followings'] = test_data.apply(lambda x: indirectFollowings(x['Source'], x['Sink']), axis=1)
test_data['Inv_Indirect_Followings'] = test_data.apply(lambda x: indirectFollowings(x['Sink'], x['Source']), axis=1)

In [38]:
test_data.to_csv("data_models/test_data.csv", sep='\t', index=False)